LSTM for sentiment analysis on 1.6 dataset

In [1]:
import os
os.chdir("/Users/imacair/Desktop/Products3/")

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.


In [19]:
data = pd.read_csv('Final_Manual_0805.csv', encoding='latin1',delimiter=',')
data.message = data.message.str.encode('ascii','replace')


In [64]:
#data= data.sample(n=10000)

In [20]:
data = data[['message','sentiment']]

In [11]:
from unidecode import unidecode

In [66]:
#data2 = pd.read_csv('Final_Manual_0805.csv', encoding='latin1',delimiter=',')
#data2 = data2[['message','sentiment']]

In [67]:
print(data[ data['sentiment'] == 0].size)
print(data[ data['sentiment'] == 4].size)

10250
9750


In [8]:
data.size

8685

In [17]:
#data['message'].values.astype('U13')

array([u'#llbite ankyl', u'#as 3 things ', u'watch physici',
       u'i dont know i', u'#imaginedrago', u'chinese medic',
       u'pain relief f', u'suffering fro', u'herself goodw',
       u'how they did ', u'imagine drago', u'the progressi', u'rt ',
       u'imagine drago', u'imagine drago', u'chinese tradi',
       u'imagine drago', u'imagine drago', u'imagine drago',
       u'imagine drago', u'great to be b', u'ankylosing sp',
       u'@doctorcaldwe', u'.@imaginedrag', u'what is #anky',
       u'how they did ', u'252.?????????', u'via @medcityn',
       u'ankylosing sp', u'my boyfriend ', u'osteoporosis ',
       u'gut microbiot', u'decreased hea', u'new2trip: tof',
       u'#plantarfasci', u'i knew there ', u'thank you @pe',
       u'the ever grow', u'exclusive: im', u'ankylosing sp',
       u'#3 mental hea', u'learn more ab', u'ankylosing sp',
       u'i need two hi', u'upper cervica', u'new ankylosin',
       u'@adamrickitt ', u'too fucking t', u'crank up the ',
       u'#nation

In [18]:
#unidecode(data['message'].values.astype('U13'))

AttributeError: 'numpy.ndarray' object has no attribute 'encode'

In [22]:
max_fatures = 2000
tokenizer = Tokenizer(nb_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['message'].values)
X = tokenizer.texts_to_sequences(data['message'].values)
X = pad_sequences(X)

In [26]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1], dropout=0.2))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(5,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

/Users/imacair/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
/Users/imacair/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, dropout=0.2, recurrent_dropout=0.2)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 128)          256000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 985       
Total params: 511,785
Trainable params: 511,785
Non-trainable params: 0
_________________________________________________________________
None


In [37]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 24)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

((646, 100), (646, 5))
((319, 100), (319, 5))


array([[   0,    0,    0, ...,    4, 1494,  507],
       [   0,    0,    0, ...,   16,    1,  286],
       [   0,    0,    0, ...,    8,   44,   47],
       ..., 
       [   0,    0,    0, ...,   55,  501,    5],
       [   0,    0,    0, ...,    5,    6,   85],
       [   0,    0,    0, ...,    4,    7, 1148]], dtype=int32)

In [38]:
batch_size = 32
model.fit(X_train, Y_train, nb_epoch = 7, batch_size=batch_size, verbose = 2)


Epoch 1/7
14s - loss: 0.6568 - acc: 0.7895
Epoch 2/7
14s - loss: 0.3985 - acc: 0.8963
Epoch 3/7
14s - loss: 0.2569 - acc: 0.9288
Epoch 4/7
13s - loss: 0.1812 - acc: 0.9474
Epoch 5/7
15s - loss: 0.1418 - acc: 0.9659
Epoch 6/7
14s - loss: 0.1003 - acc: 0.9799
Epoch 7/7
13s - loss: 0.0759 - acc: 0.9876


In [41]:
X_test

array([[   0,    0,    0, ...,  125,  599,  453],
       [   0,    0,    0, ...,    1,   16,   15],
       [   0,    0,    0, ...,   39, 1981, 1982],
       ..., 
       [   0,    0,    0, ...,  157,  420,  304],
       [   0,    0,    0, ...,    9,  110,  198],
       [   0,    0,    0, ...,  384,    5,  386]], dtype=int32)

In [42]:
validation_size = 100

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 1.09
acc: 0.72


In [59]:

#for x in range(len(X_validate)):
    
#    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]


In [74]:
print(classification_report(Y_validate,b))

             precision    recall  f1-score   support

          0       0.58      0.78      0.67         9
          1       0.84      0.70      0.76        23
          2       0.90      0.78      0.84        23
          3       0.85      0.73      0.79        15
          4       0.81      0.97      0.88        30

avg / total       0.82      0.81      0.81       100



In [73]:
b = np.zeros_like(out)
b[np.arange(len(out)), out.argmax(1)] = 1
b

array([[ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  

In [70]:
np.argmax(out[6])

2

In [47]:
out=model.predict(X_validate)

In [45]:
Y_validate

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.]])

In [45]:
from sklearn.metrics import classification_report

Link https://www.kaggle.com/ngyptr/lstm-sentiment-analysis-keras